This notebook walks through how to make graphs in this repo. You can copy this example.ipynb and rename it for your own graphing work. <b> Please do each graph in a seperate notebook!</b>

1. Import the appropriate packages. We'll use plotly.graph_objects for making all of our graphs.

In [1]:
import datetime as dt
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import utils.utils as ut
import utils.graph_templates

from fredapi import Fred

2. We'll get data using the fredapi package. Set the path to the text file with your API key

In [2]:
# Call the graph the exact same thing as its notebook (minus the ipynb suffix)
GRAPH_NAME = "wage_grwth_distr"

API_KEY_PATH = ut.get_repo_root() / "fred_api_key.txt" 

fred = Fred(api_key_file = API_KEY_PATH)

3. Set the fed_2025 template as default

In [3]:
pio.templates.default = 'fed_2025'

# Now is a good time to set the path to the graph output folder!
GRAPH_OUTPUT_PATH = ut.get_repo_root() / "figures"

4. Use the fredapi to get the data and prepare it for graphing. Documentation on the optional parameters that can be passed to the get_series called are found here (the documentation in fredapi is out of date). 

https://fred.stlouisfed.org/docs/api/fred/series_observations.html#Description

If you get data from somewhere else thats fine too! Put the raw csv in the "raw_data" folder and read it in here. Make sure not to edit the raw data, just transform and graph it.

In [4]:
# The get_series_info method may be useful
# fred.get_series_info(series_id="CPIAUCSL")

# It's good practice to store the series codes in a dictionary with their names
series_codes = {
    "PCE":"PCEPI",
    "1st to 25th Wage Percentile":"FRBATLWGT12MMUMHWGWD1WP",
    "76th to 100th Wage Percentile":"FRBATLWGT12MMUMHWGWD76WP",
    "Overall Wage Growth":"FRBATLWGT12MMUMHGO"
}

In [10]:
today = dt.date.today()

pce= fred.get_series(
    series_id=series_codes["PCE"],
    observation_start=dt.date(2017, 1, 1),
    observation_end=today,
    frequency='m'
).rename("PCE")

awg_1_to_25 = fred.get_series(
    series_id=series_codes["1st to 25th Wage Percentile"],
    observation_start=dt.date(2017, 1, 1),
    observation_end=today,
    frequency='m'
).rename("1st to 25th Wage Percentile")

awg_76_to_100=fred.get_series(
    series_id=series_codes["76th to 100th Wage Percentile"],
    observation_start=dt.date(2017, 1, 1),
    observation_end=today,
    frequency='m'
).rename("76th to 100th Wage Percentile")

awg_overall=fred.get_series(
    series_id=series_codes["Overall Wage Growth"],
    observation_start=dt.date(2017, 1, 1),
    observation_end=today,
    frequency='m'
).rename("Median 12-Month Wage Growth less PCE Inflation Rate")

pcepi_yoy = pce.pct_change(periods=12) * 100  # Gets the YoY percent change

# new YoY PCE
pce_yoy = pcepi_yoy.rename("PCE Inflation")

joined_df = pd.concat(
    [awg_overall], # srs to join
    axis=1, # multiple observations per index entry
    join='inner' # only include observations where none of the 3 are missing
)

joined_df = (joined_df.T - pce_yoy).T

new_overall=fred.get_series(
    series_id="PRS85006092",
    observation_start=dt.date(2017, 1, 1),
    observation_end=today,
    frequency='q'
).rename("Average Output per Hour, % Change Annualized")

# Naming the index makes it easier tot title
joined_df.index.name = "Date"

joined_df = pd.concat(
    [joined_df,new_overall], # srs to join
    axis=1, # multiple observations per index entry
    join='outer'
).dropna()

joined_df.tail(20)

,Median 12-Month Wage Growth less PCE Inflation Rate,"Average Output per Hour, % Change Annualized"
2020-07-01,2.677416,6.6
2020-10-01,2.412514,-3.0
2021-01-01,1.914202,3.0
2021-04-01,-0.140286,0.1
2021-07-01,-0.973216,-2.3
2021-10-01,-1.736136,2.9
2022-01-01,-2.272434,-4.9
2022-04-01,-1.984755,-3.3
2022-07-01,-1.250201,0.0
2022-10-01,-0.461602,2.7


5. Now that all our data is ready, make the graph and have it save itself as a .html file to graph_output whenver the notebooks is rerun. The name of the file should exactly match the notebook name. For instance, this file "example.ipynb" produces the graph "example.html." Nice work, you made a graph! 

In [14]:
# First make the figure
fig = go.Figure()


# Loop the columns of the dataframe and plot each as a separate trace
for col in joined_df.columns:
    fig.add_trace(
        go.Scatter(
            x=joined_df.index,
            y=joined_df[col],
            mode='lines',
            name=col
        )
    )

# Update the titles, using the html tage <sup> for a subtitle 
fig.update_layout(
    title = dict(text='Real Wages & Productivity<br><sup>Monthly, Seasonally Adjusted </sup>'),
    xaxis_title=None,
    yaxis_title=None,
)

# This is graph specific, but here we want the y-axis to be percent signs 
fig.update_yaxes(
    ticksuffix="%",
    tickformat=".2f%",
)

fig.update_layout(
    legend=dict(
        orientation = 'v'
    )
)

# Again, graph specific, we have a mutliyear series and want tick marks to be years
fig.update_xaxes(
    type='date',
    tickformat='%Y',
)

# ut.add_end_labels(fig=fig)

# Show our figure (Dimensions may be off on different screen sizes)
fig.show()

# This should be the same for EVERY GRAPH!
# Save it to the graph_output folder with the name matching the file, as HTML
fig.write_html(GRAPH_OUTPUT_PATH / f"{GRAPH_NAME}.html")